In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
import tensorflow as tf

print("--> Checking for physical Tensorflow devices")
for device in tf.config.list_physical_devices():
    print(": {}".format(device.name))

--> Checking for physical Tensorflow devices
: /physical_device:CPU:0
: /physical_device:GPU:0
: /physical_device:GPU:1


In [3]:
dfs = []

for i in range(78):
    filename = f'/kaggle/input/Traindata/Train{i}.csv'

    df = pd.read_csv(filename)
    df['ID']=filename.split('/')[-1].split('.')[0]
    dfs.append(df)

df_train = pd.concat(dfs, axis=0)

In [4]:
df_train.head()

,time,portPktIn,portPktOut,qSize,label,ID
0,0,0.0,0.0,0,8,Train0
1,1,0.0,0.0,0,8,Train0
2,2,1223516.0,1223516.0,0,8,Train0
3,3,1250452.0,1250452.0,0,8,Train0
4,4,1249416.0,1249416.0,0,8,Train0


In [5]:
#find missing data in the dataset
df_train.isnull().sum()

time          0
portPktIn     0
portPktOut    0
qSize         0
label         0
ID            0
dtype: int64

In [6]:
sample=pd.read_csv('/kaggle/input/sample/SampleSubmission.csv')

In [7]:
sample

,ID,Target
0,test0_0,0
1,test0_1,0
2,test0_2,0
3,test0_3,0
4,test0_4,0
...,...,...
2337995,test18_144995,0
2337996,test18_144996,0
2337997,test18_144997,0
2337998,test18_144998,0


In [8]:
df_train.describe()

,time,portPktIn,portPktOut,qSize,label
count,9.658000e+06,9.658000e+06,9.658000e+06,9.658000e+06,9.658000e+06
mean,6.267138e+04,9.388830e+05,9.388824e+05,8.597477e+04,5.581880e+00
std,3.704704e+04,4.891796e+05,4.864694e+05,3.822907e+05,3.444798e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,3.095500e+04,6.870650e+05,6.895868e+05,0.000000e+00,3.000000e+00
50%,6.191000e+04,1.220794e+06,1.249196e+06,1.243200e+04,6.000000e+00
75%,9.286500e+04,1.250452e+06,1.249475e+06,5.698000e+04,9.000000e+00
max,1.489990e+05,2.832285e+06,1.250741e+06,9.214954e+06,1.100000e+01


In [9]:
# train test split
from sklearn.model_selection import train_test_split

X = df_train.drop(['ID','label'], axis=1)
y = df_train['label']


In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


In [12]:

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [13]:
pca = PCA(n_components=3)  
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

In [14]:
explained_variance_ratio = pca.explained_variance_ratio_
explained_variance_ratio

array([0.51104873, 0.24958941, 0.2380361 ])

In [15]:
X_train_pca

array([[ 0.9537372 ,  0.94743808, -0.82420865],
       [ 0.08777806, -1.47630434,  0.33194136],
       [ 0.86055576, -0.34775955, -0.21667875],
       ...,
       [ 0.46783277, -1.19294631,  0.13817183],
       [-0.22144642, -0.48015605, -0.00548404],
       [ 0.77286357, -0.39037206, -0.20168182]])

In [16]:
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score

In [17]:
classifier = XGBClassifier(tree_method='gpu_hist')
model = classifier.fit(X_train_pca, y_train)

#train score
model.score(X_train_pca, y_train)

0.3050665251604887

In [18]:
y_pred = model.predict(X_test_pca)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.3041105819010147

In [19]:
dfs = []

In [20]:
for i in range(19):
    filename = f'/kaggle/input/testdata/Test{i}.csv'

    df= pd.read_csv(filename)
    df['ID']=filename.split('/')[-1].split('.')[0]
    dfs.append(df)

df_test = pd.concat(dfs, axis=0)

In [21]:
df_test.head()

,time,portPktIn,portPktOut,qSize,ID
0,0,0.0,0.0,0,Test0
1,1,371264.0,371264.0,0,Test0
2,2,0.0,0.0,0,Test0
3,3,265216.0,265216.0,0,Test0
4,4,1167329.0,1167329.0,0,Test0


In [22]:
df_test['ID'].unique()

array(['Test0', 'Test1', 'Test2', 'Test3', 'Test4', 'Test5', 'Test6',
       'Test7', 'Test8', 'Test9', 'Test10', 'Test11', 'Test12', 'Test13',
       'Test14', 'Test15', 'Test16', 'Test17', 'Test18'], dtype=object)

In [23]:
X_test=df_test.drop(['ID'],axis=1)
X_test_scaled = scaler.transform(X_test)
X_test_pca = pca.transform(X_test_scaled)

In [24]:
test_preds=model.predict(X_test_pca)
df_test['Target']=test_preds
df_test['Target']

0         0
1         5
2         0
3         4
4         6
         ..
144995    8
144996    8
144997    8
144998    8
144999    8
Name: Target, Length: 2338000, dtype: int64

In [25]:
ss=pd.read_csv('/kaggle/input/sample/SampleSubmission.csv')
ss.shape

(2338000, 2)

In [26]:
df_test['ID']=df_test['ID']+'_'+df_test['time'].astype(str)

In [27]:
df_test.head()

,time,portPktIn,portPktOut,qSize,ID,Target
0,0,0.0,0.0,0,Test0_0,0
1,1,371264.0,371264.0,0,Test0_1,5
2,2,0.0,0.0,0,Test0_2,0
3,3,265216.0,265216.0,0,Test0_3,4
4,4,1167329.0,1167329.0,0,Test0_4,6


In [28]:
df_test['ID']=df_test['ID'].str.replace('T', 't')

In [29]:
df_test.head()

,time,portPktIn,portPktOut,qSize,ID,Target
0,0,0.0,0.0,0,test0_0,0
1,1,371264.0,371264.0,0,test0_1,5
2,2,0.0,0.0,0,test0_2,0
3,3,265216.0,265216.0,0,test0_3,4
4,4,1167329.0,1167329.0,0,test0_4,6


In [31]:
sub=ss[['ID']].merge(df_test[['ID', 'Target']], how='left', on='ID')

In [32]:
sub

,ID,Target
0,test0_0,0
1,test0_1,5
2,test0_2,0
3,test0_3,4
4,test0_4,6
...,...,...
2337995,test18_144995,8
2337996,test18_144996,8
2337997,test18_144997,8
2337998,test18_144998,8


In [34]:
sub.isnull().sum()

ID        0
Target    0
dtype: int64

In [35]:
sub.to_csv('submissionfile.csv', index=False)

In [36]:
sub.shape

(2338000, 2)